# **INFERENCE**
___
___

## **INITIALIZATION**

### *IMPORTS*

In [38]:
from IPython.display import display

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import torch.nn as nn

### *DATA LOADING*

In [29]:
train_df = pd.read_csv("train.csv", index_col=0, names=["val1", "val2", "val3", "val4"])

In [30]:
display(train_df)

,val1,val2,val3,val4
0,0.012495,0.011126,0.003252,0.006625
1,0.011439,0.002691,0.001206,0.006947
2,0.000632,0.007277,0.004049,0.000074
3,0.017828,0.028210,0.007758,0.007382
4,0.021115,0.019642,0.009238,0.011499
...,...,...,...,...
741,0.001938,0.008833,0.003927,0.005106
742,0.005003,0.018943,0.003057,0.001988
743,0.007683,0.001958,0.007002,0.006467
744,0.003396,0.001280,0.007621,0.001680


___

## **DATA PREPROCESSING**

### *DATA NORMALIZATION*

In [36]:
train_df_normalized = (train_df-train_df.mean())/train_df.std()

In [37]:
display(train_df_normalized)

,val1,val2,val3,val4
0,-0.054470,-0.144799,-0.658635,-0.445863
1,-0.143146,-0.864998,-0.878996,-0.411402
2,-1.050247,-0.473369,-0.572821,-1.147324
3,0.393122,1.313894,-0.173239,-0.364766
4,0.668991,0.582378,-0.013851,0.076110
...,...,...,...,...
741,-0.940601,-0.340562,-0.585969,-0.608536
742,-0.683350,0.522619,-0.679619,-0.942462
743,-0.458394,-0.927581,-0.254648,-0.462799
744,-0.818257,-0.985469,-0.188021,-0.975389
